In [1]:
from src.data_cleaning import data_labeling

In [2]:
df = data_labeling('data/data_prepping_zeros_index.xlsx',
              'data/smalleyeball_partone.xlsx',
              'data/smalleyeball_parttwo.xlsx')

/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['spam_corporate'][df.screen_name.isin(combined_lst)] = 1
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['spam_bot'][df.expanded_url == 'https://acrobat.adobe.com'] = 1
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [4]:
sum(df['spam_corporate'])

1598